In [1]:
import xml.dom.minidom
import xml.etree.ElementTree as ET
from xml.dom.minidom import Node
import pandas as pd

In [54]:
Historical_DOU_NCH = pd.read_csv("2020 DOU Inpatient NCH.csv")
Q1 = Historical_DOU_NCH[Historical_DOU_NCH.Month.isin([1,2,3])]
Q2 = Historical_DOU_NCH[Historical_DOU_NCH.Month.isin([4,5,6])]
Q3 = Historical_DOU_NCH[Historical_DOU_NCH.Month.isin([7,8,9])]
Q4 = Historical_DOU_NCH[Historical_DOU_NCH.Month.isin([10,11,12])]

In [2]:
NCH_Data = pd.read_csv("2021_Q2 Inpatient NCH.csv")

In [3]:
set(NCH_Data.Hospital)

{'EDH', 'EHH', 'EJCH', 'ELTAC', 'ESJH', 'EUH', 'EUHM', 'EUOSH'}

In [4]:
NCH_Data

,Hospital,UnitName,Month,RNHospEmplHours,RNContractEmplHours,LPNHospEmplHours,LPNContractEmplHours,UAPHospEmplHours,UAPContractEmplHours,EMTEmplHours,ParamedicEmplHours,MHTHospEmplHours,MHTContractEmplHours
0,EHH,EHH ED,4,5059.14,1635.75,2006.86,0,1724.18,0.00,1554.48,0.0,NaN,NaN
1,EJCH,JC ED,6,4232.16,1229.57,1.65,0,1050.97,49.53,305.50,0.0,NaN,NaN
2,ESJH,ESJH Emergency Services,4,6617.17,1313.12,0.00,0,260.88,582.96,1083.76,0.0,NaN,NaN
3,EHH,EHH ED,6,5228.29,1392.78,1766.36,0,1516.78,0.00,1524.11,0.0,NaN,NaN
4,EJCH,JC ED,4,4191.29,1011.02,3.79,0,834.94,2.57,168.89,0.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
322,ESJH,3 South Telemetry (Digestive Diseases),6,3458.75,595.07,0.00,0,1138.42,82.64,NaN,NaN,NaN,NaN
323,EUH,8T N Onc,5,3334.41,665.63,0.00,0,1224.81,74.92,NaN,NaN,NaN,NaN
324,ESJH,5 West Internal Medicine,5,1918.61,1431.09,0.00,0,1230.74,61.10,NaN,NaN,NaN,NaN
325,EUOSH,OSH PACU,5,1969.80,25.68,0.00,0,149.96,0.00,NaN,NaN,NaN,NaN


In [5]:
# The variables needed for this function are the: Hospital abbreviation, Hospital NDNQI template, Hospital NDNQI
# code, Calendar year being submitted, and Calendar quarter being submitted
def NDNQI_NCH_XML(Selected_Hosp,Hosp_XML_Template,Hosp_NDNQI_Code,Calendar_Year,Calendar_Quarter):
    # This section is to update the Nursing Care Hours

    # The first step takes only the data for this selected hospital
    Hosp_data = NCH_Data[NCH_Data.Hospital == Selected_Hosp]
    # A List of this Hospital's Unit Names
    Hosp_Units = list(set(Hosp_data.UnitName))
    
    NCH_columns = ['RNHospEmplHours', 'RNContractEmplHours', 'LPNHospEmplHours', 'LPNContractEmplHours', 'UAPHospEmplHours', 'UAPContractEmplHours',
            'ParamedicEmplHours', 'EMTEmplHours']

    #This is the current XML template for this hospital and their NDNQI units for this metric
    tree = xml.dom.minidom.parse(Hosp_XML_Template)
    # This is each unit node of the xml file
    unit_nodes = tree.getElementsByTagName("UnitYrQtr")

    # Make sure the correct NDNQI code, year, and quarter are listed
    H = tree.getElementsByTagName("HospitalCode")[0].childNodes[0].nodeValue = Hosp_NDNQI_Code
    Y = tree.getElementsByTagName("Year")[0].childNodes[0].nodeValue = Calendar_Year
    Q = tree.getElementsByTagName("Quarter")[0].childNodes[0].nodeValue = Calendar_Quarter


    # This loops through each separate UnitName from the Template XML file and replaces it WITH the correct NDNQI name for the units.  
    for idx, unit in enumerate(unit_nodes):
        unit.getElementsByTagName('UnitName')[0].childNodes[0].nodeValue = Hosp_Units[idx]

        # Now grab each of the 3 months for that unit from the Hospital data
        new_data = Hosp_data[Hosp_data.UnitName == Hosp_Units[idx]]
        # Sort the data by month
        new_data = new_data.sort_values('Month')

        # Take each xml Month and replace it with each listed month from the new_data for that unit
        month_nodes = unit.getElementsByTagName("UnitMonth")
        for idx2, month in enumerate(month_nodes):
            month.getElementsByTagName('Month')[0].childNodes[0].nodeValue = new_data.Month.iloc[idx2]
            
            for data in NCH_columns:
                if data in ['ParamedicEmplHours', 'EMTEmplHours'] and pd.isna(new_data[data].iloc[idx2]):
                    month.getElementsByTagName(data)[0].childNodes[0].nodeValue = ""
                    month.getElementsByTagName(data)[0].setAttribute('xsi:nil', 'true')
                else:
                    new_value = new_data[data].iloc[idx2]
                    month.getElementsByTagName(data)[0].childNodes[0].nodeValue = new_value



    # Now, we need to write and save this XML file as its own NDNQI Quarter and Year and Hospital to upload
    # into NDNQI.
    tree.writexml(open('Q{}_{} {} Inpatient NCH.xml'.format(Calendar_Quarter,Calendar_Year,Selected_Hosp), 'w'))

## EUOSH

In [6]:
NDNQI_NCH_XML('EUOSH',"2020 EUOSH NCH Template.xml",'SBI556',2021,2)

In [7]:
EUOSH_data = NCH_Data[NCH_Data.Hospital == 'EUOSH']
# A List of this Hospital's Unit Names
EUOSH_Units = list(set(EUOSH_data.UnitName))

In [8]:
EUOSH_data

,Hospital,UnitName,Month,RNHospEmplHours,RNContractEmplHours,LPNHospEmplHours,LPNContractEmplHours,UAPHospEmplHours,UAPContractEmplHours,EMTEmplHours,ParamedicEmplHours,MHTHospEmplHours,MHTContractEmplHours
28,EUOSH,OSH ICU,6,905.75,576.97,0.0,0,0.00,0.00,NaN,NaN,NaN,NaN
44,EUOSH,OSH2,5,1596.42,2.55,0.0,0,472.59,0.00,NaN,NaN,NaN,NaN
68,EUOSH,OSH ICU,4,921.58,482.27,0.0,0,22.75,0.00,NaN,NaN,NaN,NaN
72,EUOSH,OSH4 / OSH3,4,2846.78,113.68,0.0,0,719.69,12.00,NaN,NaN,NaN,NaN
75,EUOSH,OSH POHA/PAT,4,568.32,0.00,0.0,0,144.82,0.00,NaN,NaN,NaN,NaN
113,EUOSH,OSH ICU,5,970.59,658.91,0.0,0,13.75,0.00,NaN,NaN,NaN,NaN
121,EUOSH,OSH2,4,2042.76,59.91,0.0,0,660.94,0.00,NaN,NaN,NaN,NaN
126,EUOSH,OSH PACU,4,1818.04,19.95,0.0,0,103.00,0.00,NaN,NaN,NaN,NaN
190,EUOSH,OSH2,6,1581.18,30.54,0.0,0,429.89,0.00,NaN,NaN,NaN,NaN
272,EUOSH,OSH4 / OSH3,5,3041.69,153.14,0.0,0,745.66,13.71,NaN,NaN,NaN,NaN


## EJCH

In [9]:
NDNQI_NCH_XML('EJCH','2020 EJCH NCH Template.xml','YTD069',2021,2)

In [10]:
EJCH_data = NCH_Data[NCH_Data.Hospital == 'EJCH']
# A List of this Hospital's Unit Names
EJCH_Units = list(set(EJCH_data.UnitName))

In [11]:
EJCH_data

,Hospital,UnitName,Month,RNHospEmplHours,RNContractEmplHours,LPNHospEmplHours,LPNContractEmplHours,UAPHospEmplHours,UAPContractEmplHours,EMTEmplHours,ParamedicEmplHours,MHTHospEmplHours,MHTContractEmplHours
1,EJCH,JC ED,6,4232.16,1229.57,1.65,0,1050.97,49.53,305.50,0.0,NaN,NaN
4,EJCH,JC ED,4,4191.29,1011.02,3.79,0,834.94,2.57,168.89,0.0,NaN,NaN
12,EJCH,JC ED,5,4452.43,1216.09,6.48,0,838.04,26.43,291.43,0.0,NaN,NaN
26,EJCH,JC ICU,6,5009.89,404.39,0.00,0,417.89,3.36,NaN,NaN,NaN,NaN
48,EJCH,JC ICU,5,5397.46,321.13,0.00,0,395.75,14.29,NaN,NaN,NaN,NaN
50,EJCH,JC4,4,5612.14,567.89,676.32,0,2665.96,2.57,NaN,NaN,NaN,NaN
53,EJCH,JC Neonatal ICU,6,2746.57,0.00,0.00,0,0.00,0.00,NaN,NaN,NaN,NaN
54,EJCH,JC PACU,5,1483.34,0.00,0.00,0,0.48,0.00,NaN,NaN,NaN,NaN
64,EJCH,JC Neonatal ICU,4,3265.64,0.00,0.00,0,0.00,0.00,NaN,NaN,NaN,NaN
70,EJCH,JC3,6,5922.32,647.83,281.50,0,2424.60,80.79,NaN,NaN,NaN,NaN


## ESJH

In [12]:
NDNQI_NCH_XML('ESJH','2020 ESJH NCH Template.xml','XAR317',2021,2)

In [13]:
ESJH_data = NCH_Data[NCH_Data.Hospital == 'ESJH']
# A List of this Hospital's Unit Names
ESJH_Units = list(set(ESJH_data.UnitName))

In [14]:
ESJH_data

,Hospital,UnitName,Month,RNHospEmplHours,RNContractEmplHours,LPNHospEmplHours,LPNContractEmplHours,UAPHospEmplHours,UAPContractEmplHours,EMTEmplHours,ParamedicEmplHours,MHTHospEmplHours,MHTContractEmplHours
2,ESJH,ESJH Emergency Services,4,6617.17,1313.12,0.0,0,260.88,582.96,1083.76,0.0,NaN,NaN
7,ESJH,ESJH Emergency Services,6,6739.04,1752.53,0.0,0,283.71,801.58,1055.75,0.0,NaN,NaN
9,ESJH,ESJH Emergency Services,5,7102.04,1686.27,0.0,0,266.57,862.07,1134.95,0.0,NaN,NaN
24,ESJH,4 East Telemetry (Neurovascular),6,3824.11,118.90,0.0,0,1250.22,45.25,NaN,NaN,NaN,NaN
29,ESJH,ESJH 7 West Complex Med-Surg,4,3140.05,408.34,0.0,0,954.94,323.08,NaN,NaN,NaN,NaN
45,ESJH,"ESJH Pre-Postoperative (PAT, Pre-Op, PSCU, Gro...",4,5439.30,659.79,0.0,0,1162.75,0.00,NaN,NaN,NaN,NaN
47,ESJH,3 East Telemetry (CV Step- down),4,3836.93,385.85,0.0,0,1180.41,64.84,NaN,NaN,NaN,NaN
52,ESJH,3 Southwest GI,4,2776.06,439.77,0.0,0,1096.04,59.07,NaN,NaN,NaN,NaN
62,ESJH,4 West Telemetry (Renal-Urology),4,3365.97,352.32,0.0,0,1206.98,109.86,NaN,NaN,NaN,NaN
66,ESJH,ESJH 7 West Complex Med-Surg,6,3246.82,523.61,0.0,0,1067.32,177.22,NaN,NaN,NaN,NaN


## EUH

In [15]:
NDNQI_NCH_XML('EUH','2020 EUH NCH Template.xml','JXE432',2021,2)

In [16]:
EUH_data = NCH_Data[NCH_Data.Hospital == 'EUH']
# A List of this Hospital's Unit Names
EUH_Units = list(set(EUH_data.UnitName))

In [17]:
EUH_data

,Hospital,UnitName,Month,RNHospEmplHours,RNContractEmplHours,LPNHospEmplHours,LPNContractEmplHours,UAPHospEmplHours,UAPContractEmplHours,EMTEmplHours,ParamedicEmplHours,MHTHospEmplHours,MHTContractEmplHours
8,EUH,EUH ED,6,8982.39,1225.29,0.0,0,1633.82,444.32,1083.18,0.0,NaN,NaN
10,EUH,EUH ED,5,10454.70,219.59,0.0,0,1425.00,350.34,1167.29,0.0,NaN,NaN
15,EUH,EUH ED,4,10338.41,99.50,0.0,0,1514.29,178.45,1221.25,0.0,NaN,NaN
22,EUH,5G Medical,6,1771.88,1135.52,0.0,0,1392.68,89.92,NaN,NaN,NaN,NaN
27,EUH,6B Rollins,5,652.70,187.10,0.0,0,404.07,221.25,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
304,EUH,WW Inpatient Unit,5,4161.34,1.45,0.0,0,1174.09,75.50,NaN,NaN,NaN,NaN
305,EUH,2D/2G ICU,4,16695.20,133.29,0.0,0,1075.91,0.00,NaN,NaN,NaN,NaN
306,EUH,8T S Hematology,4,6302.70,237.56,0.0,0,2070.82,31.38,NaN,NaN,NaN,NaN
316,EUH,6E CV,6,4705.35,640.44,0.0,0,1960.11,70.72,NaN,NaN,NaN,NaN


## EUHM

In [18]:
NDNQI_NCH_XML('EUHM','2020 EUHM NCH Template.xml','PVF155',2021,2)

In [19]:
EUHM_data = NCH_Data[NCH_Data.Hospital == 'EUHM']
# A List of this Hospital's Unit Names
EUHM_Units = list(set(EUHM_data.UnitName))

In [20]:
EUHM_data

,Hospital,UnitName,Month,RNHospEmplHours,RNContractEmplHours,LPNHospEmplHours,LPNContractEmplHours,UAPHospEmplHours,UAPContractEmplHours,EMTEmplHours,ParamedicEmplHours,MHTHospEmplHours,MHTContractEmplHours
11,EUHM,EUHM ED,4,6712.63,3461.07,0.0,0,3637.50,0.00,551.61,0.0,NaN,NaN
13,EUHM,EUHM ED,5,8045.67,3401.77,0.0,0,3920.24,23.50,741.59,0.0,NaN,NaN
17,EUHM,EUHM ED,6,7757.25,5305.18,0.0,0,3556.43,15.18,566.43,0.0,NaN,NaN
21,EUHM,U62 Renal,4,2610.37,1451.33,0.0,0,1002.57,314.95,NaN,NaN,NaN,NaN
23,EUHM,72 Stroke/Neuro,4,2140.29,798.21,0.0,0,1497.57,48.54,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
288,EUHM,71 ICU,4,6153.16,778.11,0.0,0,778.87,2.57,NaN,NaN,NaN,NaN
293,EUHM,71 ICU,6,5719.82,1146.21,0.0,0,666.50,14.11,NaN,NaN,NaN,NaN
299,EUHM,U61 Med (COVID-19),4,5086.65,2203.73,0.0,0,3313.96,63.63,NaN,NaN,NaN,NaN
307,EUHM,11 ICU,5,3131.48,1419.87,0.0,0,592.75,0.79,NaN,NaN,NaN,NaN


## EDH

In [21]:
NDNQI_NCH_XML('EDH','2021 EDH NCH Template.xml','NMW346',2021,2)

In [22]:
EDH_data = NCH_Data[NCH_Data.Hospital == 'EDH']
# A List of this Hospital's Unit Names
EDH_Units = list(set(EDH_data.UnitName))

In [23]:
EDH_data.head()

,Hospital,UnitName,Month,RNHospEmplHours,RNContractEmplHours,LPNHospEmplHours,LPNContractEmplHours,UAPHospEmplHours,UAPContractEmplHours,EMTEmplHours,ParamedicEmplHours,MHTHospEmplHours,MHTContractEmplHours
5,EDH,EDH ED,5,6050.00,1768.25,1503.43,0,665.02,64.84,2448.93,0.0,NaN,NaN
6,EDH,EDH ED,6,5954.49,1733.29,1456.93,0,1065.72,31.86,2354.21,0.0,NaN,NaN
14,EDH,EDH ED,4,5396.61,3120.23,1429.18,0,513.34,31.68,2430.50,0.0,NaN,NaN
18,EDH,EDH 18,4,1539.50,57.75,0.00,0,62.07,63.57,NaN,NaN,2225.23,0.0
19,EDH,EDH 18,6,1711.47,54.00,0.00,0,51.50,143.15,NaN,NaN,2487.07,0.0


## EHH

In [24]:
NDNQI_NCH_XML('EHH','2021 EHH NCH Template.xml','WXX116',2021,2)

In [25]:
EHH_data = NCH_Data[NCH_Data.Hospital == 'EHH']
# A List of this Hospital's Unit Names
EHH_Units = list(set(EHH_data.UnitName))

In [26]:
EHH_data.head()

,Hospital,UnitName,Month,RNHospEmplHours,RNContractEmplHours,LPNHospEmplHours,LPNContractEmplHours,UAPHospEmplHours,UAPContractEmplHours,EMTEmplHours,ParamedicEmplHours,MHTHospEmplHours,MHTContractEmplHours
0,EHH,EHH ED,4,5059.14,1635.75,2006.86,0,1724.18,0.0,1554.48,0.0,NaN,NaN
3,EHH,EHH ED,6,5228.29,1392.78,1766.36,0,1516.78,0.0,1524.11,0.0,NaN,NaN
16,EHH,EHH ED,5,4987.07,1374.52,1892.67,0,1624.79,0.0,1661.75,0.0,NaN,NaN
41,EHH,EHH 4 ICU,6,61.32,199.15,0.00,0,10.29,0.0,NaN,NaN,NaN,NaN
57,EHH,EHH 4 ICU,4,437.62,831.79,0.00,0,163.82,0.0,NaN,NaN,NaN,NaN


## ELTAC

In [27]:
NDNQI_NCH_XML('ELTAC','2021 ELTAC NCH Template.xml','XLS464',2021,2)

In [28]:
ELTAC_data = NCH_Data[NCH_Data.Hospital == 'ELTAC']
# A List of this Hospital's Unit Names
ELTAC_Units = list(set(ELTAC_data.UnitName))

In [29]:
ELTAC_data

,Hospital,UnitName,Month,RNHospEmplHours,RNContractEmplHours,LPNHospEmplHours,LPNContractEmplHours,UAPHospEmplHours,UAPContractEmplHours,EMTEmplHours,ParamedicEmplHours,MHTHospEmplHours,MHTContractEmplHours
51,ELTAC,ELTAC 4/5,5,3315.97,3664.24,0.0,0,4646.25,2.25,NaN,NaN,NaN,NaN
67,ELTAC,ELTAC 4/5,4,3603.68,3013.67,0.0,0,3944.86,0.00,NaN,NaN,NaN,NaN
134,ELTAC,ELTAC 4/5,6,3095.46,3188.14,0.0,0,3866.82,18.00,NaN,NaN,NaN,NaN
167,ELTAC,ELTAC 5 ICU,4,1036.87,5.09,0.0,0,344.42,0.00,NaN,NaN,NaN,NaN
213,ELTAC,ELTAC 5 ICU,5,866.59,14.21,0.0,0,7.79,0.00,NaN,NaN,NaN,NaN
226,ELTAC,ELTAC 5 ICU,6,988.54,10.29,0.0,0,209.71,0.00,NaN,NaN,NaN,NaN
